# 타켓 마케팅이 필요한 고객 대상 상품 추천 시스템 구축
- 타켓 마케팅이 필요한 고객 정의
>분기별 매출등급의 변동을 이용한 '변동지수'를 생성하고, 이를 기준으로 마케팅이 필요한 고객(총 고객의 40%)를 선정
- 변동지수
> 직전 6분기의 각 등급의 변동폭을 이용하고, 최근값일수록 더 큰 가중치를 부여하여 점수화
- 추천 알고리즘
>고객의 거래데이터의 '제휴사-소분류' feature의 value_counts 값을 0~5 사이로 점수화하고, SVD과 SGD 적용한 surprise 라이브러리를 사용하여, 타겟 고객별 추천 아이템 선정

## 전처리 라이브러리 import

In [1]:
from matplotlib import rc
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import os
import pickle

import warnings
warnings.filterwarnings('ignore')


rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

# DataFrame 변수 List
- 경쟁사이용: competitor
- 고객DOMO: demo
- 구매상품 TR: purprd
- 멤머십여부: member
- 상품분류: prodcl
- 채널이용: channel

# Data List

In [2]:
file_list = os.listdir('../../data')
sorted(file_list)

['.DS_Store',
 '테이블명_컬럼명.png',
 '경쟁사이용.txt',
 '고객DEMO.txt',
 '구매상품TR.txt',
 '멤버십여부.txt',
 '분류.docx',
 '분류.txt',
 '분류로직.csv',
 '분류로직.xlsx',
 '상품분류.txt',
 '채널이용.txt']

## 1. 경쟁사이용 : competitor

In [3]:
# 경쟁사이용
competitor = pd.read_csv('../data/경쟁사이용.txt', encoding='cp949')
competitor.head()

고객번호 제휴사  경쟁사    이용년월
0     2   D  D02  201507
1    51   D  D01  201504
2    77   D  D02  201503
3    77   D  D02  201506
4    77   D  D02  201507

- competitor.고객번호.unique()
> len: 5881\
> 1 ~ 19377
- competitor.제휴사.unique()
> ['A', 'B', 'C', 'D']
- competitor.경쟁사.unique()
> ['A01', 'A02', 'B01', 'B02', 'C01', 'C02', 'C03', 'D01', 'D02']
- competitor.이용년월.unique()
> [201501, 201502, 201503, 201504, 201505, 201506, 201507, 201508, 201509, 201510, 201511, 201512]

## 2. 고객DEMO : demo

In [4]:
# 고객DEMO
demo = pd.read_csv('../data/고객DEMO.txt', encoding='cp949')
demo.head()

고객번호 성별    연령대   거주지역
0     1  M  60세이상   60.0
1     2  M  60세이상  100.0
2     3  M  60세이상   33.0
3     4  F  60세이상   16.0
4     5  M  60세이상  100.0

- demo.고객번호.unique()
> len : 19383
> 1~19383
- demo.성별.unique()
> ['M', 'F']
- demo.연령대.unique()
> `['19세이하', '20세~24세', '25세~29세', '30세~34세', '35세~39세',
'40세~44세', '45세~49세', '50세~54세', '55세~59세', '60세이상']`
- demo.거주지역.unique())
> len : 42\
> `[10.0, 13.0, 16.0, 20.0, 24.0, 27.0, 30.0, 33.0, 36.0, 39.0,
 43.0, 45.0, 47.0, 49.0, 52.0, 55.0, 60.0, 65.0, 69.0, 72.0,
 75.0, 79.0, 82.0, 85.0, 87.0, 100.0, 210.0, 240.0, 270.0, 300.0,
 310.0, 340.0, 360.0, 410.0, 440.0, 460.0, nan, 500.0, 540.0, 570.0,
 610.0, 630.0]`

## 3. 구매상품TR : purprd

In [5]:
# 구매상품TR
purprd = pd.read_csv('../data/구매상품TR.txt', encoding='cp949')
purprd.head()

제휴사    영수증번호  대분류코드  중분류코드    소분류코드   고객번호  점포코드      구매일자  구매시간  구매금액
0   B  8664000     15   1504  B150401  17218    44  20140222    20  2420
1   B  8664000     16   1601  B160101  17218    44  20140222    20  1070
2   B  8664000     16   1602  B160201  17218    44  20140222    20  8060
3   B  8664000     18   1803  B180301  17218    44  20140222    20  6000
4   B  8664001      5    509  B050901  17674    44  20140222    22  1120

## 4. 멤버십여부 : member

In [6]:
# 멤버심 여부
member = pd.read_csv('../data/멤버십여부.txt', encoding='cp949')
member.head()

고객번호  멤버십명    가입년월
0    11  하이마트  201512
1    21  하이마트  201506
2    37  하이마트  201306
3    43  하이마트  201403
4    44  하이마트  201411

- member.고객번호.unique()
> len: 6498\
> 11 ~ 19383
- member.멤버십명.unique()
> `['다둥이', '더영', '롭스', '하이마트']`
- member.가입년월.unique()
> `[201210, 201211, 201212,
 201301, 201302, 201303, 201304, 201305, 201306, 201307, 201308, 201309, 201310, 201311, 201312,
 201401, 201402, 201403, 201404, 201405, 201406, 201407, 201408, 201409, 201410, 201411, 201412,
 201501, 201502, 201503, 201504, 201505, 201506, 201507, 201508, 201509, 201510, 201511, 201512]`
- member.멤버십명.unique()
> `['다둥이', '더영', '롭스', '하이마트']`

## 5. 상품분류: prodcl

In [7]:
# 상품분류
prodcl = pd.read_csv('../data/상품분류.txt', encoding='cp949')
prodcl.head()

제휴사  대분류코드  중분류코드    소분류코드  중분류명  소분류명
0   A      1    101  A010101  일용잡화  위생세제
1   A      1    101  A010102  일용잡화   휴지류
2   A      1    101  A010103  일용잡화  뷰티상품
3   A      1    101  A010104  일용잡화  일용잡화
4   A      1    101  A010105  일용잡화  세트상품

- prodcl.제휴사.unique()
> ['A', 'B', 'C', 'D']
- prodcl.대분류코드.unique()
    - A사: 9
    - B사: 92
    - C사: 17
    - D사: 8
- prodcl.중분류코드.unique()
    - A사: 50
    - B사: 626
    - C사: 98
    - D사: 33
- prodcl.소분류코드.unique()
    - A사: 630
    - B사: 2624
    - C사: 987
    - D사: 154
- prodcl.중분류명.unique()
    - A사: 49
    - B사: 579
    - C사: 98
    - D사: 33
- prodcl.소분류명.unique()
    - A사: 617
    - B사: 2294
    - C사: 987
    - D사: 145

## 6. 채널이용 : channel


In [8]:
# 채널이용
channel = pd.read_csv('../data/채널이용.txt', encoding='cp949')
channel.head()

고객번호           제휴사  이용횟수
0     7  A_MOBILE/APP     4
1    14  A_MOBILE/APP     1
2    42  B_MOBILE/APP    23
3    74  A_MOBILE/APP     1
4    74  B_MOBILE/APP    30

- channel.고객번호.unique()
> len: 6766\
> 7~19382
- channel.제휴사.unique()
> `['A_MOBILE/APP', 'B_MOBILE/APP', 'B_ONLINEMALL', 'C_MOBILE/APP', 'C_ONLINEMALL', 'D_MOBILE/APP']`
- channel.이용횟수.unique()


# 전처리

## 고객DEMO : pp_demo

### 1. 거주지역 코드 -> 지역명

#### 거주지역 코드 - 지역명 맵핑
- 거주지역 코드는 신우편번호 앞 세번자릿수이다.

In [10]:
area_code = {10.: '서울특별시 강북구',
             13.: '서울특별시 도봉구',
             16.: '서울특별시 노원구',
             20.: '서울특별시 중랑구',
             24.: '서울특별시 동대문구',
             27.: '서울특별시 성북구',
             30.: '서울특별시 종로구',
             33.: '서울특별시 은평구',
             36.: '서울특별시 서대문구',
             39.: '서울특별시 마포구',
             43.: '서울특별시 용산구',
             45.: '서울특별시 중구',
             47.: '서울특별시 성동구',
             49.: '서울특별시 광진구',
             52.: '서울특별시 강동구',
             55.: '서울특별시 송파구',
             60.: '서울특별시 강남구',
             65.: '서울특별시 서초구',
             69.: '서울특별시 동작구',
             72.: '서울특별시 영등포구',
             75.: '서울특별시 강서구',
             79.: '서울특별시 양천구',
             82.: '서울특별시 구로구',
             85.: '서울특별시 금천구',
             87.: '서울특별시 관악구',
             100.: '경기도',
             210.: '인천광역시',
             240.: '강원도',
             270.: '충청북도',
             300.: '세종특별자치시',
             310.: '충청남도',
             340.: '대전광역시',
             360.: '경상북도',
             410.: '대구광역시',
             440.: '울산광역시',
             460.: '부산광역시',
             500.: '경상남도',
             540.: '전라북도',
             570.: '전라남도',
             610.: '광주광역시',
             630.: '제주특별자치도'
             }

In [11]:
# demo_area = demo['거주지역'].map(lambda x:area_code[float(x)])
demo = demo.replace({'거주지역': area_code})
demo

고객번호 성별      연령대       거주지역
0          1  M    60세이상  서울특별시 강남구
1          2  M    60세이상        경기도
2          3  M    60세이상  서울특별시 은평구
3          4  F    60세이상  서울특별시 노원구
4          5  M    60세이상        경기도
...      ... ..      ...        ...
19378  19379  F  20세~24세  서울특별시 동작구
19379  19380  F  20세~24세  서울특별시 마포구
19380  19381  F  40세~44세      부산광역시
19381  19382  F  45세~49세      부산광역시
19382  19383  F  25세~29세       경상남도

[19383 rows x 4 columns]

### 2. 거주지가 불분명한 고개들 거주지 nan 값 처리

In [12]:
porc_loc_nan = pd.merge(demo[['고객번호', '거주지역']], purprd[['고객번호', '점포코드']], on='고객번호')

for i in porc_loc_nan[porc_loc_nan.거주지역.isnull()]['고객번호'].unique():
    cust_mode_store = porc_loc_nan[porc_loc_nan.고객번호 == i].점포코드.value_counts().index[0]
    demo.loc[(demo['고객번호'] == i), '거주지역'] = porc_loc_nan[porc_loc_nan.점포코드 == cust_mode_store].거주지역.value_counts().index[0]
display(demo.isnull().sum())
demo

고객번호    0
성별      0
연령대     0
거주지역    0
dtype: int64

고객번호 성별      연령대       거주지역
0          1  M    60세이상  서울특별시 강남구
1          2  M    60세이상        경기도
2          3  M    60세이상  서울특별시 은평구
3          4  F    60세이상  서울특별시 노원구
4          5  M    60세이상        경기도
...      ... ..      ...        ...
19378  19379  F  20세~24세  서울특별시 동작구
19379  19380  F  20세~24세  서울특별시 마포구
19380  19381  F  40세~44세      부산광역시
19381  19382  F  45세~49세      부산광역시
19382  19383  F  25세~29세       경상남도

[19383 rows x 4 columns]

### pp_demo - pickle

In [13]:
# demo.to_pickle("../../DATA_preprocessed/pp_demo.pickle")
pp_demo = pd.read_pickle("./pp_demo.pickle")
pp_demo.head()

고객번호 성별    연령대       거주지역
0     1  M  60세이상  서울특별시 강남구
1     2  M  60세이상        경기도
2     3  M  60세이상  서울특별시 은평구
3     4  F  60세이상  서울특별시 노원구
4     5  M  60세이상        경기도

## 구매상품TR : pp_purprd

### 1. drop columns = ['대분류코드', '중분류코드']

In [14]:
purprd.head()

제휴사    영수증번호  대분류코드  중분류코드    소분류코드   고객번호  점포코드      구매일자  구매시간  구매금액
0   B  8664000     15   1504  B150401  17218    44  20140222    20  2420
1   B  8664000     16   1601  B160101  17218    44  20140222    20  1070
2   B  8664000     16   1602  B160201  17218    44  20140222    20  8060
3   B  8664000     18   1803  B180301  17218    44  20140222    20  6000
4   B  8664001      5    509  B050901  17674    44  20140222    22  1120

In [15]:
pp_purprd = purprd[['제휴사', '영수증번호', '고객번호', '점포코드','소분류코드', '구매일자', '구매시간', '구매금액']]
pp_purprd.head()

제휴사    영수증번호   고객번호  점포코드    소분류코드      구매일자  구매시간  구매금액
0   B  8664000  17218    44  B150401  20140222    20  2420
1   B  8664000  17218    44  B160101  20140222    20  1070
2   B  8664000  17218    44  B160201  20140222    20  8060
3   B  8664000  17218    44  B180301  20140222    20  6000
4   B  8664001  17674    44  B050901  20140222    22  1120

### 2. 구매일자 연, 월, 일 분리 

In [16]:
pp_purprd['구매_연'] = pp_purprd['구매일자'].apply(lambda x : int(str(x)[:4]))
pp_purprd['구매_월'] = pp_purprd['구매일자'].apply(lambda x : int(str(x)[4:6]))
pp_purprd['구매_일'] = pp_purprd['구매일자'].apply(lambda x : int(str(x)[6:]))
pp_purprd.head()

제휴사    영수증번호   고객번호  점포코드    소분류코드      구매일자  구매시간  구매금액  구매_연  구매_월  구매_일
0   B  8664000  17218    44  B150401  20140222    20  2420  2014     2    22
1   B  8664000  17218    44  B160101  20140222    20  1070  2014     2    22
2   B  8664000  17218    44  B160201  20140222    20  8060  2014     2    22
3   B  8664000  17218    44  B180301  20140222    20  6000  2014     2    22
4   B  8664001  17674    44  B050901  20140222    22  1120  2014     2    22

In [17]:
pp_purprd = pp_purprd[['고객번호', '제휴사', '영수증번호', '점포코드', '구매_연', '구매_월', '구매_일', '구매시간', '소분류코드', '구매금액']]
pp_purprd = pp_purprd.sort_values(by=['고객번호', '제휴사', '구매_연', '구매_월', '구매_일', '구매시간'])
pp_purprd.head(10)

고객번호 제휴사    영수증번호  점포코드  구매_연  구매_월  구매_일  구매시간    소분류코드   구매금액
1550795     1   A  1182697    12  2014     1     8    14  A010302   4800
1550796     1   A  1182697    12  2014     1     8    14  A010401  25000
1550797     1   A  1182697    12  2014     1     8    14  A010508   6500
2211557     1   A  1182778    12  2014     1     8    14  A010613   6650
770730      1   A  1212480    12  2014     1    13    13  A011003  24000
392624      1   A  1216755    12  2014     1    14    13  A010401  13500
392625      1   A  1216755    12  2014     1    14    13  A010402   5000
392626      1   A  1216755    12  2014     1    14    13  A010403   9840
392627      1   A  1216755    12  2014     1    14    13  A010404   7680
392628      1   A  1216755    12  2014     1    14    13  A010707   5000

### pp_purprd - pickle

In [18]:
# pp_purprd.to_pickle("../../DATA_preprocessed/pp_purprd.pickle")
pp_purprd = pd.read_pickle("./pp_purprd.pickle")
pp_purprd.head()

고객번호 제휴사    영수증번호  점포코드  구매_연  구매_월  구매_일  구매시간    소분류코드   구매금액
1550795     1   A  1182697    12  2014     1     8    14  A010302   4800
1550796     1   A  1182697    12  2014     1     8    14  A010401  25000
1550797     1   A  1182697    12  2014     1     8    14  A010508   6500
2211557     1   A  1182778    12  2014     1     8    14  A010613   6650
770730      1   A  1212480    12  2014     1    13    13  A011003  24000

## 상품분류 : pp_prodcl

### 1. 상품 대중분류명 생성

#### 대중분류명 맵핑
- 중분류명을 대중분류명으로 카테고리화

In [19]:
m_cat={'BABY/INFANT':'유아용품',
 'BOYS 1':'완구류',
 'BOYS 2':'완구류',
 'DIY욕실용품':'욕실용품',
 'DVD':'영상',
 'EDUCATION 1':'교육완구',
 'EDUCATION 2':'교육완구',
 'Fast Food':'즉석식품',
 'GIRLS':'여아완구',
 'H&B선물세트':'케어용품',
 'LOCAL김치채소':'채소류',
 'LOCAL잎채소':'채소류',
 'MP3/PMP':'음향기기',
 'NAVI/하이패스':'자동차용품',
 'NB구두':'패션잡화',
 'PC':'컴퓨터',
 'SPORTS/OUTDOOR':'레져',
 'TRAVEL':'여행용품',
 'TV':'대형가전',
 'VIDEOGAME':'게임류',
 '가공건식품':'공산식품',
 '가공란':'공산식품',
 '가공분말류':'공산식품',
 '가공식품':'공산식품',
 '가공양곡':'곡류',
 '가공우유':'유제품',
 '가구':'대형가구',
 '가구/인테리어샵':'종합가구',
 '가루':'공산식품',
 '가방':'패션잡화',
 '가방브랜드':'패션잡화',
 '가습기':"생활가전",
 '가정잡화':'생활용품',
 '간식':'즉석식품',
 '간식용과자':'공산식품',
 '간식조리':'즉석식품',
 '간편간식':'즉석식품',
 '간편요리':'즉석식품',
 '간편조리':'즉석식품',
 '간편조리생선':'즉석식품',
 '간편조리식품':'즉석식품',
 '간편조리행사':'즉석식품',
 '간편채소':'채소류',
 '감':'과일류',
 '감귤':'과일류',
 '감미료':'감미료',
 '개인위생용품':'위생용품',
 '건강가전':'생활가전',
 '건강기능식품':'건강기능식품',
 '건강디지털기기':'생활가전',
 '건강선물세트':'건강기능식품',
 '건강식품':'건강기능식품',
 '건과':'건과',
 '건과일':'건과',
 '건면':'공산식품',
 '건어가공':'건어류',
 '건어류':'건어류',
 '건오징어':'건어류',
 '건전지':'생활용품',
 '견과류':'건과',
 '계란':'알류',
 '계란류':'알류',
 '계절팩':'계절용품',
 '고등어/삼치':'어류',
 '고양이용품':'애완용품',
 '골프':'레져',
 '골프용품':'레져',
 '공구':'공구',
 '공기청정기':'생활가전',
 '과실과일':'과일류',
 '과자':'공산식품',
 '과채과일':'과일류',
 '과채류':'채소류',
 '과채음료':'공산식품',
 '교복':'기타의류',
 '교육':'교육완구',
 '교육보조재':'교육완구',
 '교환/보수용품':'자동차용품',
 '구강용품':'케어용품',
 '구강케어':'케어용품',
 '구기용품':'스포츠용품',
 '구두':'패션잡화',
 '구색과일':'과일류',
 '구이':'즉석식품',
 '구이찜조리':'즉석식품',
 '국내산계육':'조류',
 '국내산돈육':'돈육',
 '국내산우육':'우육',
 '국내화장품':'화장품',
 '국산게/갈치':'해산물',
 '국산과일':'과일류',
 '국산과일선물세트':'과일류',
 '굴비':'어류',
 '굴비/반건생선선물세트':'어류',
 '규격RTC':'즉석식품',
 '규격생선선물세트':'어류',
 '균일가':'다이소',
 '그릇/수저':'주방용품',
 '근채류':'채소류',
 '기기':'생활가전',
 '기능성건강':'건강기능식품',
 '기능성음료':'공산식품',
 '기능헤어케어':'케어용품',
 '기저귀':'위생용품',
 '기초화장품':'화장품',
 '기타':'기타',
 '기타구색생선':'어류',
 '기타국소고기':'우육',
 '기타수산':'수산물',
 '기타음주류':'공병/공박스',
 '기타음향기기':'음향기기',
 '기타의류':'기타의류',
 '기타조리식품':'즉석식품',
 '기타주방가전':'생활가전',
 '기타축산':'즉석식품',
 '기타회':'어류',
 '김/초밥':'즉석식품',
 '김가공품':'해조류',
 '김선물세트':'해조류',
 '김치냉장고':'대형가전',
 '김치류':'김치류',
 '김치채소':'채소류',
 '김치채소류':'채소류',
 '껌':'공산식품',
 '꽁치/가자미':'어류',
 '꿀':'감미료',
 '나물':'채소류',
 '나물류':'채소류',
 '낚시':'레져',
 '남성':'패션잡화',
 '남성 트랜디':'남성의류',
 '남성구두':'패션잡화',
 '남성베이직':'남성의류',
 '남성브랜드':'남성의류',
 '남성스포츠':'스포츠의류',
 '남성스포츠NPB':'스포츠의류',
 '남성스포츠PB':'스포츠의류',
 '남성스포츠특정':'스포츠의류',
 '남성시즌행사':'기타의류',
 '남성양말':'기타의류',
 '남성어덜트캐주얼':'남성의류',
 '남성용품':'케어용품',
 '남성이지캐주얼':'남성의류',
 '남성정장':'남성의류',
 '남성캐주얼':'남성의류',
 '남성케어':'케어용품',
 '남성행사':'남성의류',
 '남아':'완구류',
 '냉동HMR':'즉석식품',
 '냉동MS':'즉석식품',
 '냉동가공':'즉석식품',
 '냉동간식':'공산식품',
 '냉동과일':'과일류',
 '냉동규격생선':'어류',
 '냉동규격해물':'해산물',
 '냉동대용식':'즉석식품',
 '냉동만두':'즉석식품',
 '냉동수산물':'해산물',
 '냉장간식MS':'공산식품',
 '냉장간편식HMR':'즉석식품',
 '냉장고':'대형가전',
 '냉장규격생선':'어류',
 '냉장기타MS':'즉석식품',
 '냉장농산':'즉석식품',
 '냉장면류':'즉석식품',
 '냉장반찬':'즉석식품',
 '냉장수산':'즉석식품',
 '냉장식사HMR':'즉석식품',
 '냉장식사MS':'즉석식품',
 '냉장음료':'공산식품',
 '냉장축산':'가공육',
 '냉장햄세트':'가공육',
 '네일메이크업':'케어용품',
 '노트':'문구류',
 '노트/지류':'문구류',
 '농산물':'농산물',
 '단무지류':'공산식품',
 '닭고기':'조류',
 '담배':'담배',
 '대구':'어류',
 '대용식':'즉석식품',
 '대중선어':'어류',
 '대형가전':'대형가전',
 '덧신':'기타의류',
 '데일리처분':'데일리처분',
 '덴탈케어':'케어용품',
 '도미/민어':'어류',
 '도시락':'즉석식품',
 '동물병원':'애완용품',
 '돼지고기':'돈육',
 '두발용품':'케어용품',
 '두부':'공산식품',
 '두유':'공산식품',
 '두채류':'채소류',
 '드럭스토어':'드럭스토어',
 '드레스화브랜드':'패션잡화',
 '등산':'레져',
 '디자이너':'디자이너',
 '디저트':'공산식품',
 '딸기':'과일류',
 '떡':'즉석식품',
 '라면':'공산식품',
 '라켓스포츠':'스포츠용품',
 '란제리':'여성의류',
 '란제리/내의':'여성의류',
 '란제리브랜드':'여성의류',
 '레몬':'과일류',
 '레스토랑':'레스토랑',
 '레저취미':'애완용품',
 '롤러보드':'스포츠용품',
 '립메이크업':'화장품',
 '마담/어덜트':'여성의류',
 '마른김':'해조류',
 '마른안주':'건어류',
 '마른안주선물세트':'공산식품',
 '만두':'즉석식품',
 '맛집행사':'레스토랑',
 '망고':'과일류',
 '매트':'생활용품',
 '맥주':'주류',
 '먹거리':'레스토랑',
 '먹거리행사':'즉석식품',
 '메론':'과일류',
 '면도용품':'케어용품',
 '면류':'공산식품',
 '멸치':'건어류',
 '멸치건명태':'건어류',
 '멸치황태미역선물세트':'건어류',
 '명품':'명품',
 '모자':'패션잡화',
 '모피/피혁':'명품',
 '묵':'즉석식품',
 '묵/즉석참기름':'즉석식품',
 '문구/팬시':'서적',
 '문화용품':'문구류',
 '물티슈':'생활용품',
 '미국산소고기':'우육',
 '미술용품':'문구류',
 '미시캐주얼':'여성의류',
 '미역/다시마':'해조류',
 '미용소품':'케어용품',
 '미용잡화':'케어용품',
 '미용티슈':'케어용품',
 '미입점행사':'미입점행사',
 '믹서기':'생활가전',
 '밀대/걸레':'생활용품',
 '바나나':'과일류',
 '바디용품':'케어용품',
 '바디케어':'케어용품',
 '반건생선':'건어류',
 '반찬':'즉석식품',
 '반찬류':'즉석식품',
 '발효유':'공산식품',
 '밤':'과일류',
 '밥류':'즉석식품',
 '밥솥':'생활가전',
 '밥죽류':'즉석식품',
 '방향/제습/탈취제':'생활용품',
 '배':'과일류',
 '버섯':'채소류',
 '버섯류':'채소류',
 '베이비케어':'유아용품',
 '베이스메이크업':'화장품',
 '베이직케어':'케어용품',
 '베이커리':'즉석식품',
 '벨트':'패션잡화',
 '벽지류':'생활용품',
 '병통조림':'공산식품',
 '병행수입':'병행수입',
 '보석':'귀금속',
 '보수용품':'공구',
 '복숭아':'과일류',
 '복합샵':'SPA',
 '복합조미료':'감미료',
 '분식':'즉석식품',
 '분유':'유아용품',
 '브랜드':'기타의류',
 '브랜드돼지고기':'돈육',
 '브랜드빵':'즉석식품',
 '브랜드조리':'즉석식품',
 '브랜드침구':'침구류',
 '브랜드푸드몰':'레스토랑',
 '브랜드한우':'우육',
 '블라우스':'여성의류',
 '블루베리':'과일류',
 '비스켓':'공산식품',
 '비즈니스정장':'남성의류',
 '비타민':'건강기능식품',
 '빵':'즉석식품',
 '뿌리채소':'채소류',
 '사과':'과일류',
 '살충제':'생활용품',
 '상':'소형가구',
 '새우':'해산물',
 '색조화장품':'화장품',
 '샌들슬리퍼':'패션잡화',
 '샐러드/간편채소':'즉석식품',
 '샐러드채소':'채소류',
 '생리대':'위생용품',
 '생리용품':'위생용품',
 '생선선물세트':'어류',
 '생선초밥':'즉석식품',
 '생선회':'어류',
 '생수':'공산식품',
 '생활소형가전':'생활가전',
 '생활잡화':'생활용품',
 '생활한방':'기타식품',
 '서적':'서적',
 '석류':'과일류',
 '선물세트':'선물세트',
 '선식':'즉석식품',
 '선식류':'즉석식품',
 '선어가공':'해산물',
 '선어류':'어류',
 '선케어':'케어용품',
 '선풍기':'생활가전',
 '섬유유연제':'케어용품',
 '섬유잡화':'패션잡화',
 '성인내의':'기타의류',
 '성인침구':'침구류',
 '성인침구류':'침구류',
 '세차용품':'자동차용품',
 '세탁기':'대형가전',
 '세탁세제':'케어용품',
 '세탁용품':'생활용품',
 '셔츠':'남성의류',
 '소가구':'소형가구',
 '소금/설탕류':'감미료',
 '소스류':'공산식품',
 '소시지':'가공육',
 '소주':'주류',
 '소형가전':'생활가전',
 '소형음향기기':'음향기기',
 '수납용품':'생활용품',
 '수도용품':'생활용품',
 '수박':'과일류',
 '수산품':'해산물',
 '수영복':'레져',
 '수예소품':'생활용품',
 '수예시즌':'생활용품',
 '수입갈치':'어류',
 '수입감귤과일':'과일류',
 '수입게':'해산물',
 '수입고등어':'어류',
 '수입과일':'과일류',
 '수입과자':'공산식품',
 '수입기타과일':'과일류',
 '수입기타생선':'어류',
 '수입돈육':'돈육',
 '수입돼지고기':'돈육',
 '수입브랜드':'수입브랜드',
 '수입연어':'어류',
 '수입우육':'우육',
 '수입육':'우육',
 '수입육선물세트':'가공육',
 '수입음료':'공산식품',
 '수입조미':'감미료',
 '수입포도':'과일류',
 '수족관용품':'레져',
 '순대':'즉석식품',
 '슈즈용품':'케어용품',
 '슈즈행사':'패션잡화',
 '스낵':'공산식품',
 '스카프':'패션잡화',
 '스키':'레져',
 '스킨케어':'케어용품',
 '스타킹':'기타의류',
 '스트리트편집샵':'편집샵',
 '스포츠':'스포츠의류',
 '스포츠ACC':'스포츠용품',
 '스포츠NB':'패션잡화',
 '스포츠브랜드편집':'패션잡화',
 '스포츠슈즈':'패션잡화',
 '스포츠웨어':'스포츠의류',
 '스포츠의류':'스포츠의류',
 '시계':'시계',
 '시니어건강용품':'생활용품',
 '시니어위생용품':'케어용품',
 '시리얼':'공산식품',
 '시즌/건강가전':'계절용품',
 '시즌반찬':'즉석식품',
 '시즌슈즈':'패션잡화',
 '시즌잡화':'패션잡화',
 '시즌차':'공산식품',
 '시즌토탈':'시즌토탈',
 '시즌행사':'시즌행사',
 '시티웨어':'기타의류',
 '시티캐주얼':'스포츠의류',
 '식기':'주방용품',
 '식사류':'즉석식품',
 '식용유':'공산식품',
 '식용유지':'공산식품',
 '식자재':'공산식품',
 '썬글라스':'패션잡화',
 '씨리얼':'공산식품',
 '아동':'유아용품',
 '아동내의':'유아용품',
 '아동복합샵':'유아용품',
 '아동브랜드':'유아용품',
 '아동슈즈':'유아용품',
 '아동양말':'유아용품',
 '아동침구':'침구류',
 '아동침구류':'침구류',
 '아동행사':'유아용품',
 '아동화브랜드':'유아용품',
 '아로마':'케어용품',
 '아웃도어퍼니처':'레져',
 '아이메이크업':'화장품',
 '아이스크림':'공산식품',
 '악기':'악기',
 '안전용품':'생활용품',
 '안주':'공산식품',
 '애견용품':'애완용품',
 '애완/동물용품':'애완용품',
 '액상조미료':'감미료',
 '액세서리':'패션잡화',
 '야외/시즌용품':'레져',
 '야외용품':'생활용품',
 '약초':'기타식품',
 '양곡':'곡류',
 '양념류':'채소류',
 '양념육':'즉석식품',
 '양념채소':'채소류',
 '양말':'기타의류',
 '양말류':'기타의류',
 '양말선물세트':'기타의류',
 '양식':'레스토랑',
 '양주':'주류',
 '양채류':'채소류',
 '어묵맛살':'즉석식품',
 '언더웨어행사':'기타의류',
 '에어컨':'대형가전',
 '여성':'패션잡화',
 '여성NPB':'여성의류',
 '여성PB':'여성의류',
 '여성구두':'패션잡화',
 '여성베이직':'여성의류',
 '여성양말':'기타의류',
 '여성용품':'위생용품',
 '여성위생용품':'위생용품',
 '여성이지캐주얼':'여성의류',
 '여성캐주얼브랜드':'여성의류',
 '여성특정':'여성의류',
 '여행용품':'여행용품',
 '연체':'연체류',
 '열대과일':'과일류',
 '열대과일기타':'과일류',
 '열대과일선물세트':'과일류',
 '열매채소':'채소류',
 '엽채류':'채소류',
 '영상':'영상',
 '예능학용품':'교육완구',
 '오디오':'음향기기',
 '오렌지':'과일류',
 '오렌지과일기타':'과일류',
 '오렌지과일선물세트':'과일류',
 '오리고기':'조류',
 '오징어':'연체류',
 '오토바이용품':'레져',
 '오피스용품':'문구류',
 '온라인규격/간편조리생선':'어류',
 '온라인돼지고기':'돈육',
 '온라인버섯/열매채소':'채소류',
 '온라인양념/뿌리채소':'채소류',
 '온라인양말':'기타의류',
 '온라인잎/샐러드채소':'채소류',
 '온라인잡곡선식':'곡류',
 '온라인저장과일':'과일류',
 '온라인해물연체':'연체류',
 '온라인화장품':'화장품',
 '와인':'주류',
 '요구르트':'유제품',
 '욕실용품':'생활용품',
 '우산/양산':'생활용품',
 '우산/양산류':'생활용품',
 '우유':'유제품',
 '우육':'우육',
 '운동화':'패션잡화',
 '워터스포츠':'레져',
 '원두커피':'공산식품',
 '원예용품':'기타',
 '웨딩':'기타',
 '위생용품':'위생용품',
 '윈터스포츠':'레져',
 '유/아동서적':'유아용품',
 '유가공':'공산식품',
 '유아동NPB':'유아용품',
 '유아동PB':'유아용품',
 '유아동용품':'유아용품',
 '유아동특정':'유아용품',
 '유아브랜드':'유아용품',
 '유아세제':'유아용품',
 '유아식기':'유아용품',
 '유아용품':'유아용품',
 '유아위생용품':'위생용품',
 '유아토들러':'유아용품',
 '유제품':'공산식품',
 '육가공품':'가공육',
 '육류':'육류',
 '육포':'가공육',
 '음료':'공산식품',
 '음반':'음반',
 '음반영상':'영상',
 '음식저장/보관':'생활용품',
 '음식조리':'주방용품',
 '의류잡화':'패션잡화',
 '의자':'소형가구',
 '의자/상':'소형가구',
 '이동통신':'기타',
 '이벤트조리':'즉석식품',
 '이온음료':'공산식품',
 '이유식/유아간식':'유아용품',
 '이지캐주얼브랜드':'편집샵',
 '인스턴트선물세트':'공산식품',
 '인스턴트커피':'공산식품',
 '일반서적':'서적',
 '일반욕실용품':'욕실용품',
 '일반의약외품':'일반의약외품',
 '일반한우':'우육',
 '일반헤어케어':'케어용품',
 '일식':'레스토랑',
 '일용잡화':'생활용품',
 '잎채소':'채소류',
 '자동차 액세서리':'자동차용품',
 '자동차 정비용품':'자동차용품',
 '자동차 청소용품':'자동차용품',
 '자동차 편의용품':'자동차용품',
 '자동차방향제/소취제':'자동차용품',
 '자동차장식용품':'자동차용품',
 '자동차처분':'유아용품',
 '자두':'과일류',
 '자몽':'과일류',
 '자전거':'레져',
 '잡곡':'곡류',
 '잡곡류':'곡류',
 '잡화멀티샵':'편집샵',
 '장갑':'패션잡화',
 '장류':'감미료',
 '장식용품':'생활용품',
 '장착용품':'기타',
 '잼':'공산식품',
 '전구':'생활용품',
 '전기면도기':'생활가전',
 '전기요/장판':'계절용품',
 '전기용품':'생활용품',
 '전기전자':'생활가전',
 '전기포트':'생활가전',
 '전자/전기':'명품',
 '전자악기':'악기',
 '전자학습기':'교육완구',
 '전통간식':'즉석식품',
 '전통과자':'즉석식품',
 '전통주':'주류',
 '전화기/팩스':'생활가전',
 '젓갈/반찬':'즉석식품',
 '젓갈류':'즉석식품',
 '정수기':'생활가전',
 '제기용품':'기타',
 '제습제':'케어용품',
 '조개':'해산물',
 '조개살':'해산물',
 '조기/명태':'어류',
 '조리기':'생활가전',
 '조리도구':'주방용품',
 '조리반찬':'즉석식품',
 '조리용기':'주방용품',
 '조명/전구':'생활용품',
 '조명기구':'생활가전',
 '조미김':'공산식품',
 '조미료':'감미료',
 '조미선물세트':'감미료',
 '조미오징어':'건어류',
 '족발':'즉석식품',
 '종량제봉투':'기타',
 '종합과자':'공산식품',
 '주거세제':'케어용품',
 '주니어스포츠':'스포츠의류',
 '주니어양말':'기타의류',
 '주니어웨어':'아동의류',
 '주류':'주류',
 '주방가전':'생활가전',
 '주방세제':'케어용품',
 '주방소형가전':'생활가전',
 '주방시즌/균일가':'주방용품',
 '주방용품':'주방용품',
 '주방정리':'주방용품',
 '주방주거세제':'케어용품',
 '주유소':'기타',
 '준보석':'귀금속',
 '준보석/시계':'시계',
 '중식':'레스토랑',
 '쥐치포':'건어류',
 '즉석구이김':'해조류',
 '즉석구이안주':'즉석식품',
 '즉석도정미':'곡류',
 '즉석식품':'즉석식품',
 '즉석참기름':'감미료',
 '지갑':'패션잡화',
 '지갑/벨트':'패션잡화',
 '집중케어':'케어용품',
 '차':'공산식품',
 '차/커피':'공산식품',
 '차류':'공산식품',
 '차별화돼지고기':'돈육',
 '차선물세트':'공산식품',
 '차음료':'공산식품',
 '참외':'과일류',
 '천연조미료':'감미료',
 '청소기':'생활가전',
 '청소욕실용품':'생활용품',
 '청소용품':'생활용품',
 '청소편의도구':'생활용품',
 '체리':'과일류',
 '초콜렛':'공산식품',
 '초콜릿':'공산식품',
 '축산가공':'유제품',
 '치즈':'유제품',
 '치즈버터':'유제품',
 '친환경과일':'과일류',
 '친환경채소':'채소류',
 '침구/수예':'침구류',
 '카메라/캠코더':'생활가전',
 '카시트/매트':'자동차용품',
 '카페':'레스토랑',
 '캐주얼':'편집샵',
 '캐주얼슈즈':'패션잡화',
 '캐주얼화브랜드':'패션잡화',
 '캔디':'공산식품',
 '캔디껌류':'공산식품',
 '캠핑':'레져',
 '커리어':'기타',
 '커튼류':'생활용품',
 '커피':'공산식품',
 '커피류':'공산식품',
 '커피메이커/머신':'생활가전',
 '커피선물세트':'공산식품',
 '커피음료':'공산식품',
 '커피차음료':'공산식품',
 '컨세션':'기타',
 '컴퓨터주변기기':'컴퓨터',
 '케첩마요네즈':'공산식품',
 '콩나물':'채소류',
 '쿠키케이크':'공산식품',
 '클렌징':'케어용품',
 '키위':'과일류',
 '키위/석류':'공산식품',
 '키친타올':'주방용품',
 '탄산음료':'공산식품',
 '탈취제':'케어용품',
 '테이블':'대형가구',
 '토마토':'과일류',
 '통신/컴퓨터':'컴퓨터',
 '통조림':'공산식품',
 '튀김':'즉석식품',
 '튀김조리':'즉석식품',
 '트래디셔널':'편집샵',
 '특산물':'즉석식품',
 '특산물행사':'건어류',
 '파이':'공산식품',
 '파인애플':'과일류',
 '파티/팬시':'문구류',
 '파티/팬시/시즌':'기타',
 '파티/팬시용품':'기타',
 '판매시설':'기타',
 '패브릭욕실용품':'욕실용품',
 '패스트푸드':'레스토랑',
 '편의시설':'기타',
 '포도':'과일류',
 '포도/체리':'과일류',
 '포장반찬':'즉석식품',
 '포장반찬HMR':'즉석식품',
 '포장젓갈':'즉석식품',
 '표백제':'케어용품',
 '푸드코트':'레스토랑',
 '풋/핸드케어':'케어용품',
 '프리미엄과자':'공산식품',
 '프리미엄매트':'생활용품',
 '피자':'레스토랑',
 '피트니스':'헬스케어',
 '피혁브랜드':'패션잡화',
 '피혁잡화':'패션잡화',
 '피혁토탈':'패션잡화',
 '필기도구':'문구류',
 '한과':'공산식품',
 '한방식재류':'기타식품',
 '한방차':'기타식품',
 '한식':'레스토랑',
 '한우선물세트':'우육',
 '해물류':'해산물',
 '해물선물세트':'해산물',
 '해물연체류':'해산물',
 '해조류':'해조류',
 '해초':'해조류',
 '해초류':'해조류',
 '핸드/풋케어':'케어용품',
 '핸드백':'패션잡화',
 '햄':'가공육',
 '햄소시지':'가공육',
 '행사':'패션잡화',
 '향신료':'감미료',
 '헤어스타일링':'케어용품',
 '헤어케어':'케어용품',
 '헬스용품':'헬스케어',
 '호주산소고기':'우육',
 '홈시어터':'대형가전',
 '홈웨어':'기타의류',
 '홍인삼':'공산식품',
 '화장지':'생활용품',
 '화장지/티슈':'생활용품',
 '화장품':'화장품',
 '화장품브랜드샵':'화장품',
 '황태':'건어류',
 '훼이셜케어':'케어용품',
 '휴대폰용품':'생활용품',
 '휴지통':'생활용품',
 '흰우유':'유제품',
 '히터기':'계절용품'}

#### 대중분류명 생성

In [20]:
prodcl.head()

제휴사  대분류코드  중분류코드    소분류코드  중분류명  소분류명
0   A      1    101  A010101  일용잡화  위생세제
1   A      1    101  A010102  일용잡화   휴지류
2   A      1    101  A010103  일용잡화  뷰티상품
3   A      1    101  A010104  일용잡화  일용잡화
4   A      1    101  A010105  일용잡화  세트상품

In [21]:
pp_prodcl = prodcl.copy()
pp_prodcl['대중분류명'] = pp_prodcl['중분류명']
pp_prodcl = pp_prodcl.replace({'대중분류명':m_cat})
pp_prodcl['제휴사_중분류명'] = pp_prodcl['제휴사'].str.cat(pp_prodcl['중분류명'])
pp_prodcl = pp_prodcl.drop(columns=['대분류코드','중분류코드'])
pp_prodcl = pp_prodcl[['대중분류명', '제휴사', '중분류명', '제휴사_중분류명', '소분류코드', '소분류명']]
pp_prodcl.head()

대중분류명 제휴사  중분류명 제휴사_중분류명    소분류코드  소분류명
0  생활용품   A  일용잡화    A일용잡화  A010101  위생세제
1  생활용품   A  일용잡화    A일용잡화  A010102   휴지류
2  생활용품   A  일용잡화    A일용잡화  A010103  뷰티상품
3  생활용품   A  일용잡화    A일용잡화  A010104  일용잡화
4  생활용품   A  일용잡화    A일용잡화  A010105  세트상품

### 2. 상품 대분류명 생성

#### 대분류명 맵핑
- 위에서 생성한 대중분류명을 새로운 대분류명으로 카테고리화

In [22]:
b_cat = {
    '식품':['가공육','감미료','건강기능식품','건과','건어류','곡류','공산식품','과일류','기타식품',\
          '김치류','농산물','돈육','레스토랑','수산물','알류','어류','연체류','우육','유제품','육류',\
          '조류','주류','즉석식품','채소류','해산물','해조류'],
    '일상용품':['계절용품','공구','문구류','생활용품','애완용품','여아완구','완구류','욕실용품','위생용품','유아용품',\
            '자동차용품','주방용품','침구류','케어용품','헬스케어','화장품'],
    '의약품/의료기기':['일반의약외품'],
    '교육/문화용품':['교육완구','서적','악기','영상','음반'],
    '디지털/가전':['게임류','대형가전','생활가전','음향기기','컴퓨터'],
    '가구/인테리어':['대형가구','소형가구','종합가구'],
    '의류':['기타의류','남성의류','여성의류','디자이너','수입브랜드','아동의류'],
    '전문스포츠/레저':['레져','스포츠용품','스포츠의류','여행용품'],
    '패션잡화':['SPA','귀금속','명품','시계','패션잡화','편집샵'],
    '기타상품':['공병/공박스','기타','다이소','담배','데일리처분','드럭스토어','미입점행사','병행수입','선물세트','시즌토탈','시즌행사']
    }

In [23]:
def search_b_cat(x):
    for i in b_cat.keys():
        if x in b_cat[i]:
            return i

#### 대분류명 생성

In [24]:
pp_prodcl['대분류명'] = pp_prodcl['대중분류명'].apply(lambda x : search_b_cat(x))
pp_prodcl=pp_prodcl[['대분류명', '대중분류명', '제휴사', '중분류명', '제휴사_중분류명', '소분류코드', '소분류명']]
pp_prodcl.head()

대분류명 대중분류명 제휴사  중분류명 제휴사_중분류명    소분류코드  소분류명
0  일상용품  생활용품   A  일용잡화    A일용잡화  A010101  위생세제
1  일상용품  생활용품   A  일용잡화    A일용잡화  A010102   휴지류
2  일상용품  생활용품   A  일용잡화    A일용잡화  A010103  뷰티상품
3  일상용품  생활용품   A  일용잡화    A일용잡화  A010104  일용잡화
4  일상용품  생활용품   A  일용잡화    A일용잡화  A010105  세트상품

### 3. 상품 type 분류 (편의품, 선매품, 전문품)

#### TYPE 맵핑
- 대중분류명을 기준으로 {'편의품', '선매품', '전문품'} 분류

In [25]:
type_cat = {
'편의품':['가공육','감미료','건강기능식품','건과','건어류','곡류','공산식품','과일류','기타식품',\
      '김치류','농산물','돈육','레스토랑','수산물','알류','어류','연체류','우육','유제품','육류',\
      '조류','주류','즉석식품','채소류','해산물','해조류','계절용품','공구','문구류','생활용품',\
       '애완용품','여아완구','완구류','욕실용품','위생용품','유아용품',\
        '자동차용품','주방용품','침구류','케어용품','헬스케어','화장품','교육완구','서적',\
      '기타의류','남성의류','여성의류','SPA',\
      '공병/공박스','기타','다이소','담배','데일리처분','드럭스토어','미입점행사','병행수입','시즌토탈','시즌행사'],

'선매품':['악기','영상','음반','일반의약외품','게임류','대형가전','생활가전','음향기기','컴퓨터','대형가구',\
       '소형가구','종합가구','레져','스포츠용품','스포츠의류','여행용품','패션잡화','편집샵','선물세트'],

'전문품':['귀금속','명품','시계','디자이너','수입브랜드']
}

In [26]:
def search_type(x):
    for i in type_cat.keys():
        if x in type_cat[i]:
            return i

In [27]:
pp_prodcl['TYPE'] = pp_prodcl['대중분류명'].apply(lambda x : search_type(x))
pp_prodcl.head()

대분류명 대중분류명 제휴사  중분류명 제휴사_중분류명    소분류코드  소분류명 TYPE
0  일상용품  생활용품   A  일용잡화    A일용잡화  A010101  위생세제  편의품
1  일상용품  생활용품   A  일용잡화    A일용잡화  A010102   휴지류  편의품
2  일상용품  생활용품   A  일용잡화    A일용잡화  A010103  뷰티상품  편의품
3  일상용품  생활용품   A  일용잡화    A일용잡화  A010104  일용잡화  편의품
4  일상용품  생활용품   A  일용잡화    A일용잡화  A010105  세트상품  편의품

##  전처리를 위한 merge : 상품 관련: purprd_prodcl
- 거래 데이터 + 상품 분류 데이터

In [28]:
purprd_prodcl = pd.merge(pp_purprd, pp_prodcl)
purprd_prodcl = purprd_prodcl.sort_values(by=['고객번호', '구매_연', '구매_월', '구매_일', '구매시간', '영수증번호']).reset_index(drop=True)
purprd_prodcl

고객번호 제휴사    영수증번호  점포코드  구매_연  구매_월  구매_일  구매시간    소분류코드   구매금액  \
0             1   A  1182697    12  2014     1     8    14  A010302   4800   
1             1   A  1182697    12  2014     1     8    14  A010401  25000   
2             1   A  1182697    12  2014     1     8    14  A010508   6500   
3             1   A  1182778    12  2014     1     8    14  A010613   6650   
4             1   A  1212480    12  2014     1    13    13  A011003  24000   
...         ...  ..      ...   ...   ...   ...   ...   ...      ...    ...   
28593025  19383   D   240712    31  2015    12    25    13  D020501   2850   
28593026  19383   D   240696    31  2015    12    25    21  D040213    600   
28593027  19383   D   240796    31  2015    12    30    23  D040106   6000   
28593028  19383   D   240796    31  2015    12    30    23  D040202   1820   
28593029  19383   D   240825    31  2015    12    31    23  D040106   1600   

          대분류명 대중분류명  중분류명 제휴사_중분류명    소분류명 TYPE  
0           식품   유제품  축산가공    A축산가공     유제품  편의품  
1           식품   농산물   농산물     A농산물      청과  편의품  
2           식품   해산물   수산품     A수산품     멸치류  편의품  
3           식품  공산식품  가공식품    A가공식품  전문베이커리  편의품  
4           식품  공산식품  차/커피    A차/커피    원두커피  편의품  
...        ...   ...   ...      ...     ...  ...  
28593025  일상용품  위생용품  여성용품    D여성용품     생리대  편의품  
28593026    식품  공산식품    음료      D음료   일반차음료  편의품  
28593027    식품  공산식품    과자      D과자    하드캔디  편의품  
28593028    식품  공산식품    음료      D음료     탄산수  편의품  
28593029    식품  공산식품    과자      D과자    하드캔디  편의품  

[28593030 rows x 16 columns]

## 벌크 추출
- 벌크 정의
> B to B로 예상되는 거래로 구매 빈도가 낮고, 같은 카테고리 내 평균 거래액보다 특정수준 이상 높은 거래 

### 고객번호별 영수증 grouping
- 벌크 거래를 하는 고객을 분류하기 위함
> 거래시 상품가짓수와 결제건당_평균금액 확인

In [29]:
bills_by_cusno = (purprd_prodcl.groupby(['고객번호', '제휴사']).영수증번호.unique())
bills_by_cusno = bills_by_cusno.reset_index()
bills_by_cusno['결제건수'] = bills_by_cusno['영수증번호'].map(lambda x: len(x))
bills_by_cusno['총구매금액'] = purprd_prodcl[['고객번호', '제휴사', '구매금액']
                                        ].groupby(['고객번호', '제휴사']).sum().reset_index()['구매금액']
purprd_prodcl[['고객번호', '영수증번호', '제휴사']].groupby(
    ['고객번호', '제휴사']).count().reset_index().rename(columns={'영수증번호': '상품가짓수'})
bills_by_cusno['상품가짓수'] = purprd_prodcl[['고객번호', '영수증번호', '제휴사']].groupby(
    ['고객번호', '제휴사']).count().reset_index().rename(columns={'영수증번호': '상품가짓수'}).상품가짓수
bills_by_cusno['결제건당_가짓수'] = bills_by_cusno['상품가짓수']/bills_by_cusno['결제건수']
bills_by_cusno['결제건당_평균금액'] = bills_by_cusno['총구매금액']/bills_by_cusno['결제건수']
bills_by_cusno = bills_by_cusno.sort_values(by='결제건당_평균금액', ascending=False)
bills_by_cusno

고객번호 제휴사                                              영수증번호  결제건수  \
43589  14323   A  [1277303, 1416069, 1500785, 1677262, 1827588, ...    17   
24341   7957   B  [6834978, 6834990, 6834991, 7010365, 7010366, ...   272   
32059  10487   C  [491092, 491093, 502111, 502112, 502113, 50211...    47   
48623  15999   A  [1167182, 1167183, 1187247, 1187248, 1195969, ...   634   
44425  14599   A  [1199553, 1266010, 1331405, 1443980, 1443981, ...   476   
...      ...  ..                                                ...   ...   
26949   8817   D                                           [245209]     1   
25237   8248   D                                           [245129]     1   
7021    2291   C                                          [1034807]     1   
16427   5350   D                                           [249320]     1   
34418  11269   C                                          [6031331]     1   

            총구매금액  상품가짓수  결제건당_가짓수     결제건당_평균금액  
43589   430757920     31  1.823529  2.533870e+07  
24341  2127884366    758  2.786765  7.823104e+06  
32059   320544290     53  1.127660  6.820091e+06  
48623  3570905490    741  1.168770  5.632343e+06  
44425  2639781570    498  1.046218  5.545760e+06  
...           ...    ...       ...           ...  
26949         300      1  1.000000  3.000000e+02  
25237         300      1  1.000000  3.000000e+02  
7021          250      1  1.000000  2.500000e+02  
16427         250      1  1.000000  2.500000e+02  
34418          20      1  1.000000  2.000000e+01  

[58631 rows x 8 columns]

### 전문품 제외
- 단일 상품의 가격이 월등하게 높은 물품을 구매하는 경우가 벌크 거래 분류를 위한 지수에 외곡을 줄 수 있어 제외

In [30]:
purprd_prodcl_without_luxury = purprd_prodcl[(purprd_prodcl.TYPE!='전문품')]
purprd_prodcl_without_luxury.sort_values(by='구매금액', ascending=False)

고객번호 제휴사     영수증번호  점포코드  구매_연  구매_월  구매_일  구매시간    소분류코드  \
11030248   7801   A   5526155     1  2015    11    20    13  A010901   
25217192  17129   A   3569101     4  2015     1    23    18  A050308   
11447253   8079   A   2220608     1  2014     6    27    10  A020303   
12630159   8888   A   4434191     1  2015     6     1    18  A040212   
12630160   8888   A   4434192     1  2015     6     1    18  A040212   
...         ...  ..       ...   ...   ...   ...   ...   ...      ...   
28264916  19096   C   9466684   531  2015     9    16    18  C010206   
28264915  19096   C   9466683   531  2015     9    16    18  C010206   
19707497  13476   B  10699136   123  2014     6     6    17  B180201   
28209430  19052   C  10384920   448  2015     7    25    17  C170701   
22916560  15629   C   6998419   194  2015     6     6    12  C170427   

              구매금액  대분류명 대중분류명   중분류명 제휴사_중분류명      소분류명 TYPE  
11030248  93654000    식품    주류     주류      A주류        주류  편의품  
25217192  93166350  패션잡화   편집샵  트래디셔널   A트래디셔널  타미힐피거상품군  선매품  
11447253  82800000  일상용품   화장품    화장품     A화장품    색조 화장품  편의품  
12630159  69046650  패션잡화   편집샵    캐주얼     A캐주얼     수입브릿지  선매품  
12630160  69046650  패션잡화   편집샵    캐주얼     A캐주얼     수입브릿지  선매품  
...            ...   ...   ...    ...      ...       ...  ...  
28264916        10    식품   과일류   수입과일    C수입과일       바나나  편의품  
28264915        10    식품   과일류   수입과일    C수입과일       바나나  편의품  
19707497         6  일상용품  생활용품    화장지     B화장지       롤티슈  편의품  
28209430         2  일상용품  생활용품   가정잡화    C가정잡화   생활잡화균일가  편의품  
22916560         1  패션잡화  패션잡화   의류잡화    C의류잡화  여성헤어액세서리  선매품  

[28470309 rows x 16 columns]

### 벌크일 가능성이 높은 고객, 거래 유추
- 결제건당_평균금액 상위 2%(201369원 이상) & 결제건당_가짓수 평균 3개 이하

#### 고객번호별 영수증 grouping without_luxury

In [32]:
bills_by_cusno_without_luxury = (purprd_prodcl_without_luxury.groupby(['고객번호', '제휴사']).영수증번호.unique())
bills_by_cusno_without_luxury = bills_by_cusno_without_luxury.reset_index()
bills_by_cusno_without_luxury['결제건수'] = bills_by_cusno_without_luxury['영수증번호'].map(lambda x: len(x))
bills_by_cusno_without_luxury['총구매금액'] = purprd_prodcl_without_luxury[['고객번호', '제휴사', '구매금액']
                                        ].groupby(['고객번호', '제휴사']).sum().reset_index()['구매금액']
bills_by_cusno_without_luxury['상품가짓수'] = purprd_prodcl_without_luxury[['고객번호', '영수증번호', '제휴사']].groupby(
    ['고객번호', '제휴사']).count().reset_index().rename(columns={'영수증번호': '상품가짓수'}).상품가짓수
bills_by_cusno_without_luxury['결제건당_가짓수'] = bills_by_cusno_without_luxury['상품가짓수']/bills_by_cusno_without_luxury['결제건수']
bills_by_cusno_without_luxury['결제건당_평균금액'] = bills_by_cusno_without_luxury['총구매금액']/bills_by_cusno_without_luxury['결제건수']
bills_by_cusno_without_luxury = bills_by_cusno_without_luxury.sort_values(by='결제건당_평균금액', ascending=False)
bills_by_cusno_without_luxury

고객번호 제휴사                                              영수증번호  결제건수  \
24339   7957   B  [6834978, 6834990, 6834991, 7010365, 7010366, ...   272   
32053  10487   C  [491092, 491093, 502111, 502112, 502113, 50211...    47   
48608  15999   A  [1167182, 1167183, 1187247, 1187248, 1195969, ...   629   
44413  14599   A  [1199553, 1266010, 1331405, 1443980, 1443981, ...   475   
35884  11763   C   [5806733, 10826860, 10827018, 10827019, 5858541]     5   
...      ...  ..                                                ...   ...   
25235   8248   D                                           [245129]     1   
26947   8817   D                                           [245209]     1   
16426   5350   D                                           [249320]     1   
7021    2291   C                                          [1034807]     1   
34410  11269   C                                          [6031331]     1   

            총구매금액  상품가짓수  결제건당_가짓수     결제건당_평균금액  
24339  2127884366    758  2.786765  7.823104e+06  
32053   320544290     53  1.127660  6.820091e+06  
48608  3565775890    736  1.170111  5.668960e+06  
44413  2639755770    497  1.046316  5.557381e+06  
35884    22689920      5  1.000000  4.537984e+06  
...           ...    ...       ...           ...  
25235         300      1  1.000000  3.000000e+02  
26947         300      1  1.000000  3.000000e+02  
16426         250      1  1.000000  2.500000e+02  
7021          250      1  1.000000  2.500000e+02  
34410          20      1  1.000000  2.000000e+01  

[58610 rows x 8 columns]

#### 결제건당_평균금액 상위 2%(21369원 이상) 고객 선별

In [33]:
np.percentile(bills_by_cusno_without_luxury['결제건당_평균금액'], 98)

201369.4570907774

In [34]:
b_to_b_cust = bills_by_cusno_without_luxury[bills_by_cusno_without_luxury['결제건당_평균금액'] >= np.percentile(bills_by_cusno_without_luxury['결제건당_평균금액'], 98)] # 98%는 주관의 영역입니다.
b_to_b_cust

고객번호 제휴사                                              영수증번호  결제건수  \
24339   7957   B  [6834978, 6834990, 6834991, 7010365, 7010366, ...   272   
32053  10487   C  [491092, 491093, 502111, 502112, 502113, 50211...    47   
48608  15999   A  [1167182, 1167183, 1187247, 1187248, 1195969, ...   629   
44413  14599   A  [1199553, 1266010, 1331405, 1443980, 1443981, ...   475   
35884  11763   C   [5806733, 10826860, 10827018, 10827019, 5858541]     5   
...      ...  ..                                                ...   ...   
45764  15049   A  [1162284, 1162298, 1169905, 1174909, 1174910, ...   403   
4900    1600   A  [1259569, 1426385, 3949249, 4288242, 4288258, ...     7   
10247   3335   A  [1175814, 1175859, 1175968, 1175970, 1185363, ...   438   
6603    2156   A  [1144965, 1153167, 1153183, 1153184, 1183254, ...   525   
45212  14866   A  [1345317, 1345318, 1345319, 1345334, 1345265, ...   545   

            총구매금액  상품가짓수  결제건당_가짓수     결제건당_평균금액  
24339  2127884366    758  2.786765  7.823104e+06  
32053   320544290     53  1.127660  6.820091e+06  
48608  3565775890    736  1.170111  5.668960e+06  
44413  2639755770    497  1.046316  5.557381e+06  
35884    22689920      5  1.000000  4.537984e+06  
...           ...    ...       ...           ...  
45764    81196310    666  1.652605  2.014797e+05  
4900      1410100      7  1.000000  2.014429e+05  
10247    88230320    532  1.214612  2.014391e+05  
6603    105722710    776  1.478095  2.013766e+05  
45212   109746360    556  1.020183  2.013695e+05  

[1173 rows x 8 columns]

#### 결제건당_가짓수 평균 3개 이하 

In [35]:
# 벌크를 구매했을 가능성이 있는 고객 번호
b_to_b_cust = b_to_b_cust[b_to_b_cust['결제건당_가짓수'] <= 3].고객번호.unique()
b_to_b_cust = sorted(b_to_b_cust)
len(b_to_b_cust)

1071

### 벌크로 구매했을 가능성이 있는 고객 거래중 벌크 데이터 분류
- 위 거래중 B to B 거래를 하는 고객 이지만 소매거래를 하는 경우의 거래 내역을 분류
> 건당 구매금액이 10만원 이하인 거래는 소매거래로 판단

In [36]:
b_to_b_cust_tr = purprd_prodcl_without_luxury[purprd_prodcl_without_luxury['고객번호'].isin(b_to_b_cust)]
b_to_b_cust_tr

고객번호 제휴사    영수증번호  점포코드  구매_연  구매_월  구매_일  구매시간    소분류코드    구매금액  \
9252          9   C  6976265   189  2014     1     2    11  C120501   10000   
9253          9   C  6976265   189  2014     1     2    11  C150701    1100   
9254          9   C  6976265   189  2014     1     2    11  C150301    5980   
9255          9   A  1151025     7  2014     1     3    14  A010521   34000   
9256          9   A  1246068     7  2014     1    19    14  A060203  600780   
...         ...  ..      ...   ...   ...   ...   ...   ...      ...     ...   
28576920  19365   B  7156991    13  2015    12    30    10  B010106    2080   
28576921  19365   B  7156991    13  2015    12    30    10  B010202    1500   
28576922  19365   B  7156991    13  2015    12    30    10  B480301    4662   
28576923  19365   B  7157071    13  2015    12    30    15  B270301    8900   
28576924  19365   B  7157071    13  2015    12    30    15  B270302   15400   

              대분류명 대중분류명     중분류명  제휴사_중분류명      소분류명 TYPE  
9252            식품    주류      전통주      C전통주       막걸리  편의품  
9253            식품  즉석식품     전통과자     C전통과자    기타전통과자  편의품  
9254            식품  공산식품    쿠키케이크    C쿠키케이크       크래커  편의품  
9255            식품   해산물      수산품      A수산품    수산단기행사  편의품  
9256      전문스포츠/레저    레져     골프용품     A골프용품  직수입 골프의류  선매품  
...            ...   ...      ...       ...       ...  ...  
28576920        식품   채소류     열매채소     B열매채소       애호박  편의품  
28576921        식품   채소류       버섯       B버섯     느타리버섯  편의품  
28576922        식품    돈육  브랜드돼지고기  B브랜드돼지고기    NB돼지고기  편의품  
28576923      일상용품  애완용품     애견용품     B애견용품      애견간식  편의품  
28576924      일상용품  애완용품     애견용품     B애견용품    애견건식사료  편의품  

[1353705 rows x 16 columns]

#### 벌크로 추정되는 구매 금액이 10만원 이상인 거래

In [37]:
b_to_b_recipe = b_to_b_cust_tr[(b_to_b_cust_tr['구매금액'] >= 100000)].영수증번호.unique()
print(len(b_to_b_recipe))
b_to_b_recipe

143998


array([1246068, 1246069, 1246046, ..., 7147552, 5596323, 7154428])

#### 상기 조건에 해당하는 TR의 index

In [38]:
purprd_prodcl[purprd_prodcl['영수증번호'].isin(b_to_b_recipe)].index

Int64Index([    9256,     9257,     9258,     9259,     9260,     9262,
                9266,     9267,     9268,     9273,
            ...
            28576216, 28576361, 28576422, 28576423, 28576579, 28576624,
            28576703, 28576711, 28576800, 28576876],
           dtype='int64', length=155976)

### TR 데이터에서 벌크 거래 내역 삭제

In [39]:
purprd_prodcl_without_bulk = purprd_prodcl.drop(index=purprd_prodcl[purprd_prodcl['영수증번호'].isin(b_to_b_recipe)].index)
purprd_prodcl_without_bulk

고객번호 제휴사    영수증번호  점포코드  구매_연  구매_월  구매_일  구매시간    소분류코드   구매금액  \
0             1   A  1182697    12  2014     1     8    14  A010302   4800   
1             1   A  1182697    12  2014     1     8    14  A010401  25000   
2             1   A  1182697    12  2014     1     8    14  A010508   6500   
3             1   A  1182778    12  2014     1     8    14  A010613   6650   
4             1   A  1212480    12  2014     1    13    13  A011003  24000   
...         ...  ..      ...   ...   ...   ...   ...   ...      ...    ...   
28593025  19383   D   240712    31  2015    12    25    13  D020501   2850   
28593026  19383   D   240696    31  2015    12    25    21  D040213    600   
28593027  19383   D   240796    31  2015    12    30    23  D040106   6000   
28593028  19383   D   240796    31  2015    12    30    23  D040202   1820   
28593029  19383   D   240825    31  2015    12    31    23  D040106   1600   

          대분류명 대중분류명  중분류명 제휴사_중분류명    소분류명 TYPE  
0           식품   유제품  축산가공    A축산가공     유제품  편의품  
1           식품   농산물   농산물     A농산물      청과  편의품  
2           식품   해산물   수산품     A수산품     멸치류  편의품  
3           식품  공산식품  가공식품    A가공식품  전문베이커리  편의품  
4           식품  공산식품  차/커피    A차/커피    원두커피  편의품  
...        ...   ...   ...      ...     ...  ...  
28593025  일상용품  위생용품  여성용품    D여성용품     생리대  편의품  
28593026    식품  공산식품    음료      D음료   일반차음료  편의품  
28593027    식품  공산식품    과자      D과자    하드캔디  편의품  
28593028    식품  공산식품    음료      D음료     탄산수  편의품  
28593029    식품  공산식품    과자      D과자    하드캔디  편의품  

[28437054 rows x 16 columns]

## 상품 Lev 맵핑
- 상품 분류 대분류별, 소분류별 평균을 비교하여 상대적인 {'고가', '중가', '저가'} 여부를 판단

### 대중분류의 평균 - without bulk -pickle

In [40]:
# 대중분류의 평균
대중분류별_평균 = pd.pivot_table(data=purprd_prodcl_without_bulk,index=['대중분류명'], values='구매금액',aggfunc=(np.mean))
대중분류별_평균 = 대중분류별_평균.reset_index()
대중분류별_평균 = 대중분류별_평균.rename(columns={'구매금액':'대중분류별_평균'})
# 대중분류별_평균.to_pickle("../../DATA_preprocessed/대중분류별_평균.pickle")
대중분류별_평균 = pd.read_pickle("./대중분류별_평균.pickle")
대중분류별_평균

대중분류명       대중분류별_평균
0      SPA   28330.147337
1      가공육    6036.649354
2      감미료    6091.879478
3   건강기능식품   90565.732158
4       건과    8946.659344
..     ...            ...
78     편집샵  130261.258953
79     해산물   13761.729316
80     해조류    4431.231404
81    헬스케어   18133.030505
82     화장품   69210.716728

[83 rows x 2 columns]

### 소분류의 평균 - without bulk - pickle

In [41]:
# 소분류의 평균
소분류별_평균 = pd.pivot_table(data=purprd_prodcl_without_bulk,index=['소분류명'], values='구매금액',aggfunc=(np.mean))
소분류별_평균=소분류별_평균.reset_index()
소분류별_평균=소분류별_평균.rename(columns={'구매금액':'소분류별_평균'})
# 소분류별_평균.to_pickle("../../DATA_preprocessed/소분류별_평균.pickle")
소분류별_평균 = pd.read_pickle("./소분류별_평균.pickle")
소분류별_평균

소분류명        소분류별_평균
0        14K  185551.113720
1       2단우산   10617.385621
2       3단우산    8788.340426
3      3분요리류    3599.729149
4     4대 B/D  272402.144538
...      ...            ...
3509     흑미류   15489.862321
3510   흰다리새우    8861.810915
3511      히터   25317.187500
3512     히터기  147000.000000
3513    힐앤토트  612193.362832

[3514 rows x 2 columns]

In [42]:
pp_prodcl = pd.merge(pp_prodcl, 대중분류별_평균, how='left')
pp_prodcl = pd.merge(pp_prodcl, 소분류별_평균, how='left')
pp_prodcl

대분류명 대중분류명 제휴사    중분류명 제휴사_중분류명    소분류코드      소분류명 TYPE      대중분류별_평균  \
0     일상용품  생활용품   A    일용잡화    A일용잡화  A010101      위생세제  편의품   9692.805436   
1     일상용품  생활용품   A    일용잡화    A일용잡화  A010102       휴지류  편의품   9692.805436   
2     일상용품  생활용품   A    일용잡화    A일용잡화  A010103      뷰티상품  편의품   9692.805436   
3     일상용품  생활용품   A    일용잡화    A일용잡화  A010104      일용잡화  편의품   9692.805436   
4     일상용품  생활용품   A    일용잡화    A일용잡화  A010105      세트상품  편의품   9692.805436   
...    ...   ...  ..     ...      ...      ...       ...  ...           ...   
4381  일상용품   화장품   D  아이메이크업  D아이메이크업  D080204     아이섀도우  편의품  69210.716728   
4382  일상용품   화장품   D  아이메이크업  D아이메이크업  D080205    메이크업세트  편의품  69210.716728   
4383  일상용품   화장품   D   립메이크업   D립메이크업  D080301  립스틱/립라이너  편의품  69210.716728   
4384  일상용품   화장품   D   립메이크업   D립메이크업  D080302   립글로즈/틴트  편의품  69210.716728   
4385  일상용품  케어용품   D  네일메이크업  D네일메이크업  D080401  일반네일/케어류  편의품   9503.312803   

            소분류별_평균  
0      11534.323855  
1       4777.805556  
2      18480.475486  
3       6099.166201  
4     197789.296703  
...             ...  
4381   19909.785523  
4382   13042.415459  
4383   16695.196970  
4384   17050.603448  
4385   10243.036304  

[4386 rows x 10 columns]

## 대중분류별_평균/소분류별_평균 컬럼 생성

In [43]:
# purprd_prodcl = purprd_prodcl.drop(columns=['중분류명','제휴사_중분류명'])
pp_prodcl['소분류별_평균/대중분류별_평균'] = round(pp_prodcl['소분류별_평균']/pp_prodcl['대중분류별_평균'],4)
pp_prodcl['대중분류별_평균'] = round(pp_prodcl['대중분류별_평균'],2)
pp_prodcl['소분류별_평균'] = round(pp_prodcl['소분류별_평균'],2)
pp_prodcl.head()

대분류명 대중분류명 제휴사  중분류명 제휴사_중분류명    소분류코드  소분류명 TYPE  대중분류별_평균    소분류별_평균  \
0  일상용품  생활용품   A  일용잡화    A일용잡화  A010101  위생세제  편의품   9692.81   11534.32   
1  일상용품  생활용품   A  일용잡화    A일용잡화  A010102   휴지류  편의품   9692.81    4777.81   
2  일상용품  생활용품   A  일용잡화    A일용잡화  A010103  뷰티상품  편의품   9692.81   18480.48   
3  일상용품  생활용품   A  일용잡화    A일용잡화  A010104  일용잡화  편의품   9692.81    6099.17   
4  일상용품  생활용품   A  일용잡화    A일용잡화  A010105  세트상품  편의품   9692.81  197789.30   

   소분류별_평균/대중분류별_평균  
0            1.1900  
1            0.4929  
2            1.9066  
3            0.6292  
4           20.4058

### lev 컬럼 생성

In [44]:
def define_lev(x):
    if x > 2:
        return '고가'
    elif x > 0.5:
        return '중가'
    else:
        return '저가'
pp_prodcl['lev'] = pp_prodcl['소분류별_평균/대중분류별_평균'].apply(define_lev)
pp_prodcl = pp_prodcl[['대분류명', '대중분류명', '제휴사', '중분류명', '제휴사_중분류명', '소분류코드', '소분류명', 'TYPE', 'lev']]
pp_prodcl

대분류명 대중분류명 제휴사    중분류명 제휴사_중분류명    소분류코드      소분류명 TYPE lev
0     일상용품  생활용품   A    일용잡화    A일용잡화  A010101      위생세제  편의품  중가
1     일상용품  생활용품   A    일용잡화    A일용잡화  A010102       휴지류  편의품  저가
2     일상용품  생활용품   A    일용잡화    A일용잡화  A010103      뷰티상품  편의품  중가
3     일상용품  생활용품   A    일용잡화    A일용잡화  A010104      일용잡화  편의품  중가
4     일상용품  생활용품   A    일용잡화    A일용잡화  A010105      세트상품  편의품  고가
...    ...   ...  ..     ...      ...      ...       ...  ...  ..
4381  일상용품   화장품   D  아이메이크업  D아이메이크업  D080204     아이섀도우  편의품  저가
4382  일상용품   화장품   D  아이메이크업  D아이메이크업  D080205    메이크업세트  편의품  저가
4383  일상용품   화장품   D   립메이크업   D립메이크업  D080301  립스틱/립라이너  편의품  저가
4384  일상용품   화장품   D   립메이크업   D립메이크업  D080302   립글로즈/틴트  편의품  저가
4385  일상용품  케어용품   D  네일메이크업  D네일메이크업  D080401  일반네일/케어류  편의품  중가

[4386 rows x 9 columns]

### pp_prodcl - pickle

In [45]:
# pp_prodcl.to_pickle("../../DATA_preprocessed/pp_prodcl.pickle")
pp_prodcl = pd.read_pickle("./pp_prodcl.pickle")
print(len(pp_prodcl))
pp_prodcl.head()

4386


대분류명 대중분류명 제휴사  중분류명 제휴사_중분류명    소분류코드  소분류명 TYPE lev
0  일상용품  생활용품   A  일용잡화    A일용잡화  A010101  위생세제  편의품  중가
1  일상용품  생활용품   A  일용잡화    A일용잡화  A010102   휴지류  편의품  저가
2  일상용품  생활용품   A  일용잡화    A일용잡화  A010103  뷰티상품  편의품  중가
3  일상용품  생활용품   A  일용잡화    A일용잡화  A010104  일용잡화  편의품  중가
4  일상용품  생활용품   A  일용잡화    A일용잡화  A010105  세트상품  편의품  고가

# 상품관련 최종 merge : pp_purprd_prodcl

In [48]:
pp_purprd_prodcl = pd.merge(purprd_prodcl_without_bulk, pp_prodcl)
pp_purprd_prodcl

고객번호 제휴사    영수증번호  점포코드  구매_연  구매_월  구매_일  구매시간    소분류코드   구매금액  \
0             1   A  1182697    12  2014     1     8    14  A010302   4800   
1             1   A  1239229    12  2014     1    18    15  A010302   3500   
2             1   A  1307657    12  2014     1    30    11  A010302   3900   
3             1   A  1402358    12  2014     2    18    16  A010302   3500   
4             1   A  1456379    12  2014     2    28    15  A010302   3500   
...         ...  ..      ...   ...   ...   ...   ...   ...      ...    ...   
28437049  18697   C   631324    29  2014     5    30    21  C060405   2300   
28437050  18697   C   631629    29  2014     7    20    15  C060405   2300   
28437051  18710   C  1073027    75  2015     1    22    12  C030312   7990   
28437052  18710   C  1073138    75  2015     1    29    21  C030312   7990   
28437053  19364   D   209812    13  2015     7    11    21  D040112  10500   

         대분류명 대중분류명  중분류명 제휴사_중분류명    소분류명 TYPE lev  
0          식품   유제품  축산가공    A축산가공     유제품  편의품  중가  
1          식품   유제품  축산가공    A축산가공     유제품  편의품  중가  
2          식품   유제품  축산가공    A축산가공     유제품  편의품  중가  
3          식품   유제품  축산가공    A축산가공     유제품  편의품  중가  
4          식품   유제품  축산가공    A축산가공     유제품  편의품  중가  
...       ...   ...   ...      ...     ...  ...  ..  
28437049   식품  즉석식품  베이커리    C베이커리   페스츄리류  편의품  중가  
28437050   식품  즉석식품  베이커리    C베이커리   페스츄리류  편의품  중가  
28437051   식품   채소류   근채류     C근채류  기타한방약재  편의품  고가  
28437052   식품   채소류   근채류     C근채류  기타한방약재  편의품  고가  
28437053   식품  공산식품    과자      D과자  컵아이스크림  편의품  중가  

[28437054 rows x 17 columns]

## pp_purprd_prodcl - pickle

In [50]:
# pp_purprd_prodcl.to_pickle('./pp_purprd_prodcl.pickle')
pp_purprd_prodcl = pd.read_pickle('./pp_purprd_prodcl.pickle')
pp_purprd_prodcl

고객번호 제휴사    영수증번호  점포코드  구매_연  구매_월  구매_일  구매시간    소분류코드   구매금액  \
0             1   A  1182697    12  2014     1     8    14  A010302   4800   
1             1   A  1239229    12  2014     1    18    15  A010302   3500   
2             1   A  1307657    12  2014     1    30    11  A010302   3900   
3             1   A  1402358    12  2014     2    18    16  A010302   3500   
4             1   A  1456379    12  2014     2    28    15  A010302   3500   
...         ...  ..      ...   ...   ...   ...   ...   ...      ...    ...   
28437049  18697   C   631324    29  2014     5    30    21  C060405   2300   
28437050  18697   C   631629    29  2014     7    20    15  C060405   2300   
28437051  18710   C  1073027    75  2015     1    22    12  C030312   7990   
28437052  18710   C  1073138    75  2015     1    29    21  C030312   7990   
28437053  19364   D   209812    13  2015     7    11    21  D040112  10500   

         대분류명 대중분류명  중분류명 제휴사_중분류명    소분류명 TYPE lev  
0          식품   유제품  축산가공    A축산가공     유제품  편의품  중가  
1          식품   유제품  축산가공    A축산가공     유제품  편의품  중가  
2          식품   유제품  축산가공    A축산가공     유제품  편의품  중가  
3          식품   유제품  축산가공    A축산가공     유제품  편의품  중가  
4          식품   유제품  축산가공    A축산가공     유제품  편의품  중가  
...       ...   ...   ...      ...     ...  ...  ..  
28437049   식품  즉석식품  베이커리    C베이커리   페스츄리류  편의품  중가  
28437050   식품  즉석식품  베이커리    C베이커리   페스츄리류  편의품  중가  
28437051   식품   채소류   근채류     C근채류  기타한방약재  편의품  고가  
28437052   식품   채소류   근채류     C근채류  기타한방약재  편의품  고가  
28437053   식품  공산식품    과자      D과자  컵아이스크림  편의품  중가  

[28437054 rows x 17 columns]